---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here

In [3]:
#importing train, test and addresses data
train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
addresses_df = pd.read_csv('addresses.csv')
latlons_df = pd.read_csv('latlons.csv')
test_df = pd.read_csv('test.csv')

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train_df.compliance.fillna(value=0, inplace=True)

In [5]:
# start creating training set

Y = train_df.compliance
X = train_df.drop(columns=['compliance'])

In [6]:
# add coordinates

addresses_and_latlons = addresses_df.merge(latlons_df, how='outer', on='address')
X = X.merge(addresses_and_latlons, on='ticket_id')
test_df = test_df.merge(addresses_and_latlons, on='ticket_id')

In [7]:
# starting research wich columns to pick for a learning algorythm

all_cols = X.columns
print(len(all_cols))
print(all_cols)

36
Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'address', 'lat', 'lon'],
      dtype='object')


In [8]:
train_df[all_cols[0:15]].head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,country,ticket_issued_date
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,60606,NaN,USA,2004-03-16 11:40:00
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,48208,NaN,USA,2004-04-23 12:30:00
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,48223,NaN,USA,2004-04-26 13:40:00
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,48214,NaN,USA,2004-04-26 13:30:00
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,48206,NaN,USA,2004-04-26 13:00:00


In [9]:
train_df[all_cols[15:30]].head()

,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status
0,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED
1,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL
2,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED
3,NaN,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED
4,2005-03-29 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED


In [47]:
#test_df[all_cols[0:15]]

In [48]:
#test_df[all_cols[15:30]]

In [49]:
# number of unique values
#for col in X.columns:
#    print(col, 'coltype - ', X[col].dtype, ' - ', len(X[col].unique()))

In [50]:
print(len(X.violation_code.unique()), type(X.violation_code.unique()))
print(len(test_df.violation_code.unique()))
all_codes = np.concatenate((X.violation_code.unique(), test_df.violation_code.unique()))
print(len(all_codes))

235 <class 'numpy.ndarray'>
151
386


In [39]:
from sklearn.preprocessing import LabelEncoder

selected_cols = ['ticket_id', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
                   'judgment_amount', 'lat', 'lon', 'violation_code']
X_train_selected = pd.DataFrame(X[selected_cols])
X_test_selected = pd.DataFrame(test_df[selected_cols])

#enc1 = LabelEncoder()
#X_train_selected.violation_code = enc1.fit(np.concatenate((X_train_selected.violation_code.unique(), X_test_selected.violation_code.unique())))
#X_train_selected.violation_code = enc1.transform(X_train_selected.violation_code)
#X_test_selected.violation_code = enc1.transform(X_test_selected.violation_code)
#state_encoder = LabelEncoder()

In [40]:
X_train_selected.head()

,ticket_id,fine_amount,admin_fee,state_fee,late_fee,judgment_amount,lat,lon,violation_code
0,22056,250.0,20.0,10.0,25.0,305.0,42.390729,-83.124268,9-1-36(a)
1,27586,750.0,20.0,10.0,75.0,855.0,42.326937,-83.135118,61-63.0600
2,22062,250.0,0.0,0.0,0.0,0.0,42.380516,-83.096069,9-1-36(a)
3,22084,250.0,0.0,0.0,0.0,0.0,42.380570,-83.095919,9-1-36(a)
4,22093,250.0,0.0,0.0,0.0,0.0,42.145257,-83.208233,9-1-36(a)


In [41]:
X_test_selected.head()

,ticket_id,fine_amount,admin_fee,state_fee,late_fee,judgment_amount,lat,lon,violation_code
0,284932,200.0,20.0,10.0,20.0,250.0,42.407581,-82.986642,22-2-61
1,285362,1000.0,20.0,10.0,100.0,1130.0,42.426239,-83.238259,22-2-88(b)
2,285361,100.0,20.0,10.0,10.0,140.0,42.426239,-83.238259,22-2-43
3,285338,200.0,20.0,10.0,20.0,250.0,42.309661,-83.122426,22-2-88(b)
4,285346,100.0,20.0,10.0,10.0,140.0,42.308830,-83.121116,22-2-45


In [42]:
X_train_selected.fillna(value=0, inplace=True)
X_train_selected.isna().any()

ticket_id          False
fine_amount        False
admin_fee          False
state_fee          False
late_fee           False
judgment_amount    False
lat                False
lon                False
violation_code     False
dtype: bool

In [43]:
X_test_selected.fillna(value=0, inplace=True)
X_test_selected.isna().any()

ticket_id          False
fine_amount        False
admin_fee          False
state_fee          False
late_fee           False
judgment_amount    False
lat                False
lon                False
violation_code     False
dtype: bool

In [14]:
print(Y.shape, X_train_selected.shape, X_test_selected.shape)

(250306,) (250306, 8) (61001, 8)


In [22]:
# building model
# 1 - Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_train_selected, Y, random_state=0)

In [59]:
%%time
clf = RandomForestClassifier().fit(X_train, y_train)

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 1.61 s


In [60]:
y_predicted_rf = clf.predict(X_test)

roc_auc_score(y_test[y_test != 2.0], y_predicted_rf[y_predicted_rf != 2.0])

0.6263080770727489

In [62]:
# 2 -lightgbm
import lightgbm as lgb

%%time
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)
y_predicted_lgb = model.predict(X_test)

Wall time: 506 ms


In [64]:
# AUC score for the test set

roc_auc_score(y_test[y_test != 2.0], y_predicted_lgb[y_predicted_lgb != 2.0])

0.87560249731052

In [74]:
y_predicted_lgb_result = model.predict(X_test_selected)

result = pd.DataFrame(y_predicted_lgb_result, index=X_test_selected.ticket_id, columns=['compliance'])

In [92]:
result.compliance[result.compliance > 1.0] = 1
result.compliance[result.compliance < 0] = 0

### Sklearn Gradient Boosting

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

In [37]:
%%time
GB_clf = GradientBoostingClassifier(n_estimators=200).fit(X_train, y_train)

y_predicted = GB_clf.predict(X_test)

print('roc_auc = ', roc_auc_score(y_test, y_predicted))

print('train score = ', GB_clf.score(X_train, y_train))
print('test score = ', GB_clf.score(X_test, y_test))

roc_auc =  0.5570653887329234
train score =  0.9584827064545168
test score =  0.9571567828435368
Wall time: 8.12 s


In [45]:
%%time
GB_clf = GradientBoostingClassifier(n_estimators=200).fit(X_train, y_train)

y_predicted = GB_clf.predict(X_test)

print('roc_auc = ', roc_auc_score(y_test, y_predicted))

print('train score = ', GB_clf.score(X_train, y_train))
print('test score = ', GB_clf.score(X_test, y_test))

ValueError: could not convert string to float: '9-1-36(a)'

In [28]:
print(len(y_predicted))
print(y_predicted.sum())

62577
389.0


In [36]:
(y_predicted == y_test).sum()

59896